In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
cms_physician = pd.read_csv('../data/cms_physician_and_other_supplier-payments_2015-2017.csv')
#for line in open('u.item', encoding='ISO-8859-1'):
#read each line
cms_physician.columns

Index(['year', 'National Provider Identifier',
       'Last Name/Organization Name of the Provider',
       'First Name of the Provider', 'Middle Initial of the Provider',
       'Credentials of the Provider', 'Gender of the Provider',
       'Entity Type of the Provider', 'Street Address 1 of the Provider',
       'Street Address 2 of the Provider', 'City of the Provider',
       'Zip Code of the Provider', 'State Code of the Provider',
       'Country Code of the Provider', 'Provider Type',
       'Medicare Participation Indicator', 'Place of Service', 'HCPCS Code',
       'HCPCS Description', 'HCPCS Drug Indicator', 'Number of Services',
       'Number of Medicare Beneficiaries',
       'Number of Distinct Medicare Beneficiary/Per Day Services',
       'Average Medicare Allowed Amount', 'Average Submitted Charge Amount',
       'Average Medicare Payment Amount',
       'Average Medicare Standardized Amount'],
      dtype='object')

In [3]:
#drop columns
cms_physician = cms_physician.drop(columns=['First Name of the Provider',
                                            'Last Name/Organization Name of the Provider',
                                            'Middle Initial of the Provider',
                                            'Credentials of the Provider',
                                            'Gender of the Provider',
                                            'Entity Type of the Provider',
                                            'Street Address 1 of the Provider',
                                            'Street Address 2 of the Provider',
                                            'State Code of the Provider',
                                            'Country Code of the Provider',
                                            'Medicare Participation Indicator',
                                            'Place of Service',
                                            'HCPCS Drug Indicator',
                                            'Number of Distinct Medicare Beneficiary/Per Day Services',
                                            'Average Medicare Allowed Amount',
                                            'Average Medicare Standardized Amount'], axis=1)
cms_physician.head()

,year,National Provider Identifier,City of the Provider,Zip Code of the Provider,Provider Type,HCPCS Code,HCPCS Description,Number of Services,Number of Medicare Beneficiaries,Average Submitted Charge Amount,Average Medicare Payment Amount
0,2017,1033112180,CHATTANOOGA,374211688,Maxillofacial Surgery,70355,X-ray imaging of teeth,25,25,104.400000,11.348000
1,2017,1043302466,NASHVILLE,372320001,Diagnostic Radiology,70355,X-ray imaging of teeth,27,26,80.000000,7.824074
2,2017,1104009646,KNOXVILLE,379201502,Maxillofacial Surgery,21248,Partial reconstruction of lower jaw or cheek b...,30,19,1970.000000,662.898333
3,2017,1104009646,KNOXVILLE,379201502,Maxillofacial Surgery,70355,X-ray imaging of teeth,60,60,85.833333,13.701667
4,2017,1104031053,NASHVILLE,372328605,Otolaryngology,21461,Open treatment of broken jaw bone,11,11,3614.000000,348.310000


In [4]:
#rename columns
cms_physician = cms_physician.rename(columns = {'National Provider Identifier':'provider_id',
                                                'City of the Provider':'city',
                                                'Zip Code of the Provider':'postal_code',
                                                'City of the Provider':'city',
                                                'State Code of the Provider':'state',
                                                'Provider Type':'treatment_type',
                                                'HCPCS Code':'hcpcs_code',
                                                'HCPCS Description':'description',
                                                'Number of Services':'services_performed',
                                                'Number of Medicare Beneficiaries':'beneficiaries',
                                                'Average Submitted Charge Amount':'avg_charge_submitted',
                                                'Average Medicare Payment Amount':'avg_medicare payment'
                                               })
cms_physician.head()

,year,provider_id,city,postal_code,treatment_type,hcpcs_code,description,services_performed,beneficiaries,avg_charge_submitted,avg_medicare payment
0,2017,1033112180,CHATTANOOGA,374211688,Maxillofacial Surgery,70355,X-ray imaging of teeth,25,25,104.400000,11.348000
1,2017,1043302466,NASHVILLE,372320001,Diagnostic Radiology,70355,X-ray imaging of teeth,27,26,80.000000,7.824074
2,2017,1104009646,KNOXVILLE,379201502,Maxillofacial Surgery,21248,Partial reconstruction of lower jaw or cheek b...,30,19,1970.000000,662.898333
3,2017,1104009646,KNOXVILLE,379201502,Maxillofacial Surgery,70355,X-ray imaging of teeth,60,60,85.833333,13.701667
4,2017,1104031053,NASHVILLE,372328605,Otolaryngology,21461,Open treatment of broken jaw bone,11,11,3614.000000,348.310000


In [5]:
#export to csv
cms_physician.to_csv('../cms_physician.csv', index = False)

In [6]:
cms_physician.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  93 non-null     int64  
 1   provider_id           93 non-null     int64  
 2   city                  93 non-null     object 
 3   postal_code           93 non-null     int64  
 4   treatment_type        93 non-null     object 
 5   hcpcs_code            93 non-null     int64  
 6   description           93 non-null     object 
 7   services_performed    93 non-null     int64  
 8   beneficiaries         93 non-null     int64  
 9   avg_charge_submitted  93 non-null     float64
 10  avg_medicare payment  93 non-null     float64
dtypes: float64(2), int64(6), object(3)
memory usage: 8.1+ KB


### Group table by city, year, and number of visits

In [7]:
#delete last 4 digits in postal_code
#cms_physician['postal_code'] = cms_physician['postal_code'].astype(str).str[:-4]
#cms_physician.head()

In [8]:
#change services_performed to number_of_visits to match smile_on dataframe
cms_physician = cms_physician.rename(columns = {'services_performed': 'number_of_visits'})

In [9]:
#drop unnecessary columns
cms_physician = cms_physician.drop(columns=['provider_id',
                                            'postal_code',
                                            'treatment_type',
                                            'hcpcs_code',
                                            'description',
                                            'beneficiaries',
                                            'avg_charge_submitted',
                                            'avg_medicare payment'
                                           ])

### create 2017 dataframe

In [10]:
#isolate 2017
cms_physician_2017 = cms_physician.loc[cms_physician['year'] == 2017]
cms_physician_2017.head()

,year,city,number_of_visits
0,2017,CHATTANOOGA,25
1,2017,NASHVILLE,27
2,2017,KNOXVILLE,30
3,2017,KNOXVILLE,60
4,2017,NASHVILLE,11


In [11]:
cms_physician_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              33 non-null     int64 
 1   city              33 non-null     object
 2   number_of_visits  33 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ KB


In [12]:
cms_2017_grouped = cms_physician_2017.groupby(['city']).agg({'number_of_visits': "sum"}).reset_index()
cms_2017_grouped

,city,number_of_visits
0,CHATTANOOGA,87
1,HIXSON,72
2,KNOXVILLE,403
3,MEMPHIS,34
4,NASHVILLE,382


### create 2016 dataframe

In [13]:
#isolate 2016
cms_physician_2016 = cms_physician.loc[cms_physician['year'] == 2016]
cms_physician_2016

,year,city,number_of_visits
33,2016,CHATTANOOGA,14
34,2016,NASHVILLE,26
35,2016,KNOXVILLE,34
36,2016,KNOXVILLE,14
37,2016,HIXSON,60
38,2016,CHATTANOOGA,24
39,2016,KNOXVILLE,53
40,2016,NASHVILLE,105
41,2016,NASHVILLE,63
42,2016,NASHVILLE,43


In [14]:
#export to csv
#cms_physician_2016.to_csv('../cms_physician_2016.csv', index = False)

In [15]:
cms_2016_grouped = cms_physician_2016.groupby(['city']).agg({'number_of_visits': "sum"}).reset_index()
cms_2016_grouped

,city,number_of_visits
0,CHATTANOOGA,110
1,HIXSON,79
2,KNOXVILLE,378
3,NASHVILLE,524


### merge dataframes and calculate average number of visits by city

In [16]:
cms_average = (cms_2016_grouped.set_index('city').add(cms_2017_grouped.set_index('city'), fill_value=0)*.5).reset_index()
cms_average = cms_average.rename(columns = {'number_of_visits':'cms_visits_per_year'})
cms_average

,city,cms_visits_per_year
0,CHATTANOOGA,98.5
1,HIXSON,75.5
2,KNOXVILLE,390.5
3,MEMPHIS,17.0
4,NASHVILLE,453.0


In [17]:
cms_average['city']=cms_average['city'].str.title()
cms_average

,city,cms_visits_per_year
0,Chattanooga,98.5
1,Hixson,75.5
2,Knoxville,390.5
3,Memphis,17.0
4,Nashville,453.0


### import smile on data

In [18]:
smile_on = pd.read_csv('../data/smile_on.csv', encoding="ISO-8859-1",  low_memory=False, header=2)
#for line in open('u.item', encoding='ISO-8859-1'):
#read each line
smile_on.head()

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,PostalCode,...,SMILE ON 60+ Screening - Last Screening Date,SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?,SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic,SMILE ON 60+ Oral Care Encounter - What care was provided to the enrollee? (Select all that apply),SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - can chew and ""social six"" esthetics",SMILE ON 60+ Verification of Eligibility - Transportation Arranged
0,81381827,SMILE ON 60+ Registration,11/25/2020 16:46,11/25/2020 16:48,3859874,136529,Springfield,Robertson,TN,37172.0,...,NaN,No,Matthew Walker Comprehensive Health Center Inc...,NaN,Matthew Walker Comprehensive Health Center Inc...,Provider Site,NaN,NaN,NaN,NaN
1,81379177,SMILE ON 60+ Registration,11/25/2020 15:40,11/25/2020 15:55,3859755,82341,Nashville,Davidson,TN,37212.0,...,NaN,No,Interfaith Dental Clinic - Nashville,NaN,NaN,Health Assist Office,NaN,NaN,NaN,NaN
2,81377435,SMILE ON 60+ Clinical,11/25/2020 15:02,11/25/2020 15:02,2518680,119506,Rogersville,Hawkins,TN,37857.0,...,NaN,No,NaN,5650 - Add tooth to existing partial denture - 30,Friends in Need Health Center - Kingsport,Provider Site,Yes,Yes,Yes,NaN
3,81375215,SMILE ON 60+ Clinical,11/25/2020 14:17,11/25/2020 14:17,3231923,143703,Memphis,Shelby,TN,38104.0,...,NaN,Yes,NaN,No Code - Denture step Mandibular: may be used...,Christ Community Health Services- Frayser,Provider Site,NaN,NaN,NaN,NaN
4,81373934,SMILE ON 60+ Clinical,11/25/2020 13:50,11/25/2020 13:58,-1,135611,NaN,NaN,TN,NaN,...,11/25/2020,NaN,NaN,NaN,Neighborhood Health - Lebanon,NaN,NaN,Yes,No,NaN


In [19]:
#drop irrelevant columns
smile_on = smile_on.drop(columns=['CallReportNum',
                                  'ReportVersion',
                                  'CallDateAndTimeEnd',
                                  'CallerNum',
                                  'PhoneWorkerNum',
                                  'CountyName',
                                  'StateProvince',
                                  'PostalCode',
                                  'EnteredOn',
                                  'SMILE ON 60+ Screening - Last Screening Date',
                                  'SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?',
                                  'SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic',
                                  'SMILE ON 60+ Oral Care Encounter - What care was provided to the enrollee? (Select all that apply)',
                                  'SMILE ON 60+ Site Information - Enroller ID',
                                  'SMILE ON 60+ Site Information - Enroller Location',
                                  'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?',
                                  'SMILE ON 60+ Treatment Plan - Was a treatment plan developed?',
                                  'SMILE ON 60+ Treatment Plan - can chew and "social six" esthetics',
                                  'SMILE ON 60+ Verification of Eligibility - Transportation Arranged'                                ''
                                 ])
smile_on.head()

,CallDateAndTimeStart,CityName
0,11/25/2020 16:46,Springfield
1,11/25/2020 15:40,Nashville
2,11/25/2020 15:02,Rogersville
3,11/25/2020 14:17,Memphis
4,11/25/2020 13:50,NaN


In [20]:
#extract year
smile_on['year'] = pd.DatetimeIndex(smile_on['CallDateAndTimeStart']).year
smile_on.head(25)

,CallDateAndTimeStart,CityName,year
0,11/25/2020 16:46,Springfield,2020
1,11/25/2020 15:40,Nashville,2020
2,11/25/2020 15:02,Rogersville,2020
3,11/25/2020 14:17,Memphis,2020
4,11/25/2020 13:50,NaN,2020
5,11/25/2020 13:04,Kingsport,2020
6,11/25/2020 13:03,Lebanon,2020
7,11/25/2020 13:02,Church Hill,2020
8,11/25/2020 13:00,Lebanon,2020
9,11/25/2020 11:46,Memphis,2020


In [21]:
#delete .0 in postal_code
#smile_on['CityName'] = smile_on['CityName'].astype(str).str[:-2]
#smile_on.head()

In [22]:
#drop column and change column name
smile_on = smile_on.drop(columns=['CallDateAndTimeStart'])
smile_on = smile_on.rename(columns = {'CityName': 'city'})
smile_on.head()

,city,year
0,Springfield,2020
1,Nashville,2020
2,Rogersville,2020
3,Memphis,2020
4,NaN,2020


In [23]:
#remove null values for city
smile_on = smile_on.dropna()
smile_on.head()

,city,year
0,Springfield,2020
1,Nashville,2020
2,Rogersville,2020
3,Memphis,2020
5,Kingsport,2020


### For year 2018

In [24]:
#for year 2018
smile_on_18 = smile_on.loc[smile_on['year'] == 2018].reset_index()
smile_on_18.head()

,index,city,year
0,35419,Nashville,2018
1,35420,Blountville,2018
2,35421,Lebanon,2018
3,35422,Nashville,2018
4,35423,Nashville,2018


In [25]:
smile_on_18 = smile_on_18.drop(columns=['index'])

In [26]:
smile_visits_18 = smile_on_18['city'].value_counts()
smile_visits_18

Nashville         1077
Murfreesboro       381
Memphis            311
Cleveland          287
Kingsport          168
                  ... 
Flag Pond            1
Tellico Plains       1
Charlotte            1
Gatlinburg           1
Hohenwald            1
Name: city, Length: 183, dtype: int64

In [27]:
smile_visits_18 = smile_visits_18.to_frame().reset_index()
smile_visits_18.head()

,index,city
0,Nashville,1077
1,Murfreesboro,381
2,Memphis,311
3,Cleveland,287
4,Kingsport,168


In [28]:
#rename columns
smile_visits_18 = smile_visits_18.rename(columns = {'city':'visits_18'})
smile_visits_18 = smile_visits_18.rename(columns = {'index':'city'})
smile_visits_18.head()

,city,visits_18
0,Nashville,1077
1,Murfreesboro,381
2,Memphis,311
3,Cleveland,287
4,Kingsport,168


### For year 2019

In [29]:
#for year 2019
smile_on_19 = smile_on.loc[smile_on['year'] == 2019].reset_index()
smile_on_19 = smile_on_19.drop(columns=['index'])
smile_on_19.head()

,city,year
0,Memphis,2019
1,Jackson,2019
2,Memphis,2019
3,Memphis,2019
4,Memphis,2019


In [30]:
smile_visits_19 = smile_on_19['city'].value_counts()
smile_visits_19.head()

Memphis         3164
Nashville       3151
Murfreesboro    1152
Cleveland        867
Knoxville        671
Name: city, dtype: int64

In [31]:
smile_visits_19 = smile_visits_19.to_frame().reset_index()
smile_visits_19.head()

,index,city
0,Memphis,3164
1,Nashville,3151
2,Murfreesboro,1152
3,Cleveland,867
4,Knoxville,671


In [32]:
#rename columns
smile_visits_19 = smile_visits_19.rename(columns = {'city':'visits_19'})
smile_visits_19 = smile_visits_19.rename(columns = {'index':'city'})
smile_visits_19.head()

,city,visits_19
0,Memphis,3164
1,Nashville,3151
2,Murfreesboro,1152
3,Cleveland,867
4,Knoxville,671


In [33]:
smile_average = pd.merge(smile_visits_18, smile_visits_19, left_on = 'city', right_on = 'city', how = 'outer')
smile_average.head()

,city,visits_18,visits_19
0,Nashville,1077.0,3151.0
1,Murfreesboro,381.0,1152.0
2,Memphis,311.0,3164.0
3,Cleveland,287.0,867.0
4,Kingsport,168.0,651.0


In [34]:
smile_average['avg_visits_per_year'] = (smile_average['visits_18'] + smile_average['visits_19'])*.5
smile_average.head()

,city,visits_18,visits_19,avg_visits_per_year
0,Nashville,1077.0,3151.0,2114.0
1,Murfreesboro,381.0,1152.0,766.5
2,Memphis,311.0,3164.0,1737.5
3,Cleveland,287.0,867.0,577.0
4,Kingsport,168.0,651.0,409.5


In [35]:
smile_average = smile_average.drop(columns=['visits_18','visits_19'])
smile_average = smile_average.rename(columns = {'avg_visits_per_year':'smile_visits'})
smile_average.head()

,city,smile_visits
0,Nashville,2114.0
1,Murfreesboro,766.5
2,Memphis,1737.5
3,Cleveland,577.0
4,Kingsport,409.5


### merge dataframes and calculate average number of visits by city

In [36]:
avg_yearly_visits_by_city = pd.merge(cms_average, smile_average, left_on = 'city', right_on = 'city', how = 'outer')
avg_yearly_visits_by_city.head()

,city,cms_visits_per_year,smile_visits
0,Chattanooga,98.5,7.0
1,Hixson,75.5,NaN
2,Knoxville,390.5,394.0
3,Memphis,17.0,1737.5
4,Nashville,453.0,2114.0


In [37]:
avg_yearly_visits_by_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361 entries, 0 to 360
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   city                 361 non-null    object 
 1   cms_visits_per_year  5 non-null      float64
 2   smile_visits         169 non-null    float64
dtypes: float64(2), object(1)
memory usage: 11.3+ KB


In [38]:
#change NaN values to zero
avg_yearly_visits_by_city['smile_visits'] = avg_yearly_visits_by_city['smile_visits'].fillna(0)
avg_yearly_visits_by_city['cms_visits_per_year'] = avg_yearly_visits_by_city['cms_visits_per_year'].fillna(0)
avg_yearly_visits_by_city.head(25)

,city,cms_visits_per_year,smile_visits
0,Chattanooga,98.5,7.0
1,Hixson,75.5,0.0
2,Knoxville,390.5,394.0
3,Memphis,17.0,1737.5
4,Nashville,453.0,2114.0
5,Murfreesboro,0.0,766.5
6,Cleveland,0.0,577.0
7,Kingsport,0.0,409.5
8,Smyrna,0.0,217.5
9,Johnson City,0.0,227.0


In [39]:
#export to csv
avg_yearly_visits_by_city.to_csv('../avg_yearly_visits_by_city.csv', index = False)